#### Imports

In [ ]:
import utilities.logging as Logger
import integration.files as Files
import evaluation.general as GenEval
import evaluation.iv as IVEval
import utilities.macros as Macros
import plotting.plot as Plot

#### Init

In [ ]:
bib = Files.DataCollection()

Logger.setup(bib)
Logger.set_level(Logger.INFO)
Files.setup(bib, "Test", "/home/dacap/Downloads")
GenEval.setup(bib)
IVEval.setup(bib)

#### Config

In [ ]:
bib.data.file_name = "OI-25c-09 2025-04-15 unbroken antenna full 1-formated.hdf5"
bib.params.volt_amp = (1.0, 1.0)

In [ ]:
# Files.showFile(bib.data)

#### Selection

In [ ]:
GenEval.loadMeasurements(bib)
GenEval.showLoadedMeasurements(bib)
GenEval.select_measurement(bib, 2)

IVEval.select_edge(bib, 1, "down")

In [ ]:
Macros.eval(bib, 1, "adwin")

Macros.bulk_eval(bib)

In [ ]:
# Plot.plot_curves(bib, ("cache", "adwin"), [("voltage", "current")], True)

# style = {
#     "title": "",
#     "x-axis": "voltage (V)",
#     "y-axis": "voltage (V)",
#     "z-axis": "current (A)",
# }

# Plot.plot_map(bib, ("adwin/voltage-bin", "adwin/current-voltage"), style)
# Plot.plot_map(bib, ("adwin/time-bin", "adwin/current"))